In [1]:
import generate_data as vaina
import models
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from evaluate import *


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

Sacamos el conjunto de datos de entramiento y lo dividimos en 5% del subset para entrenamiento

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #Define el dispositivo que PyTorch usará para ejecutar los cálculos.

data = vaina.Data(layer_size=6)
train, val, test = data.get_splits(['random_subsample'], [[0.05, 0.95]]) #internamente onehot
train_unshuffled = train.copy()
np.random.shuffle(train)

Number of samples: 26 train | 100 val | 403 test


C:\Users\Luna Santos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


In [ ]:
data.samples

<function list.__sizeof__()>

Separamos todos los datos en x (variables), y (clasificacion) 

In [4]:
x_train, y_train = data.get_x_y(train) #devuelve  self.np_data = self.one_hot_encode(self.samples_char_sep)
x_val, y_val = data.get_x_y(val)
x_test, y_test = data.get_x_y(test)

In [51]:
x_train[0]
#print(x_train[:10])

array([0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,
       0.0], dtype=object)

In [6]:
print(y_train[:10])


['V' 'V' 'V' 'V' 'C' 'V' 'C' 'D' 'C' 'V']


In [7]:
train_original = data.reverse_encoding(data.get_x_y(train_unshuffled)[0])
val_original = data.reverse_encoding(x_val)
test_original = data.reverse_encoding(x_test)

print("--------train_original--------", '\n')
pos_train_ratio = get_stats_and_ratio(train_original)
print("--------test_original--------", '\n')
pos_test_ratio = get_stats_and_ratio(test_original)
print("--------val_original--------", '\n')
pos_val_ratio = get_stats_and_ratio(val_original)

--------train_original-------- 

Number of samples by case:
F3 dr:    F2-wu 1 | F2-uw 2 | F2-w 1 | F2-noop 2 | 
F3 cr:    F2-wu 2 | F2-uw 1 | F2-w 3 | F2-noop 1 | 
F3 r:    F2-wu 0 | F2-uw 1 | F2-w 1 | F2-noop 4 | 
F3 noop:    F2-wu 3 | F2-uw 1 | F2-w 2 | F2-noop 1 | 

Positive samples count: 11
Total samples count: 26
Positive class ratio: 0.4230769230769231
--------test_original-------- 

Number of samples by case:
F3 dr:    F2-wu 20 | F2-uw 17 | F2-w 24 | F2-noop 22 | 
F3 cr:    F2-wu 20 | F2-uw 21 | F2-w 22 | F2-noop 27 | 
F3 r:    F2-wu 27 | F2-uw 25 | F2-w 30 | F2-noop 29 | 
F3 noop:    F2-wu 24 | F2-uw 23 | F2-w 32 | F2-noop 40 | 

Positive samples count: 174
Total samples count: 403
Positive class ratio: 0.4317617866004963
--------val_original-------- 

Number of samples by case:
F3 dr:    F2-wu 4 | F2-uw 6 | F2-w 5 | F2-noop 11 | 
F3 cr:    F2-wu 3 | F2-uw 3 | F2-w 5 | F2-noop 7 | 
F3 r:    F2-wu 3 | F2-uw 4 | F2-w 5 | F2-noop 9 | 
F3 noop:    F2-wu 8 | F2-uw 11 | F2-w 8 | F2-

Mandamos las etiquetas codificadas al dispositivo del tensor

In [ ]:
# Crear un codificador de etiquetas
label_encoder = LabelEncoder()

# Ajustar el codificador a las etiquetas y transformarlas a num enteros
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)

y_trainP = y_train

#Un tensor es una estructura de datos similar a matrices o arrays, pero con soporte para operaciones avanzadas en GPU.
# Convertir las etiquetas a tensores de PyTorch
y_train = torch.tensor(y_train, dtype=torch.float).to(device)
y_val = torch.tensor(y_val, dtype=torch.float).to(device)
y_test = torch.tensor(y_test, dtype=torch.float).to(device)

#se mueven los datos a la CPU o GPU para que puedan ser procesados por PyTorch durante el entrenamiento o la inferencia.

In [ ]:

print(set(y_train))  

{tensor(3.), tensor(1.), tensor(3.), tensor(1.), tensor(3.), tensor(0.), tensor(3.), tensor(3.), tensor(0.), tensor(3.), tensor(3.), tensor(3.), tensor(3.), tensor(3.), tensor(3.), tensor(2.), tensor(1.), tensor(1.), tensor(2.), tensor(3.), tensor(1.), tensor(2.), tensor(3.), tensor(3.), tensor(1.), tensor(3.)}


In [12]:
x_train_cnn = data.to_conv_format(x_train)
x_val_cnn = data.to_conv_format(x_val)
x_test_cnn = data.to_conv_format(x_test)
for i in range(len(x_train_cnn)): #itereamos sobre cada tensor
    x_train_cnn[i] = x_train_cnn[i].to(device)
    x_val_cnn[i] = x_val_cnn[i].to(device)
    x_test_cnn[i] = x_test_cnn[i].to(device)

In [ ]:
x_train_cnn[0].shape #devuelve un par que indica (num de muestras, numero de funciones en total = 3*numMuestras)

torch.Size([26, 48])

In [ ]:
x_train_cnn[0][2] #[x][y] accedes fx de la muestra y    max [2][25]

tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])

x_train_cnn está representado como una matriz tridimensional en donde cada fila es la función y cada columna es la muestra

In [ ]:
num_experiments = 6 #cuantas veces ejecutaremos el entrenamiento, pero con dif pesos, entrena, parame, semillas
epochs = 200 #época es una iteración completa sobre todo el conjunto de entrenamiento
early_stopping_limit = 100 #si el cjto de val no ve una mejora durante 100 epocas consecutivas, el entrenam se para. Evitamos sobreajuste

experiment_name="25per"

#el número de épocas determina cuántas veces el modelo va a ajustarse a los datos y aprender de ellos. Sin embargo, no siempre se necesita un número tan alto; por eso también se utilizan estrategias como el early stopping para evitar el sobreajuste.

In [ ]:
def train_models(constructor, x_train, x_val, x_test, weight_decay, *argv):
    accuracies = [] #lista con la precisión de cada modelo en el cjto de prueba
    precisions = []
    recalls = []
    f1_scores = []
    wrong_preds = []#lista con las predicciones incorrectas del modelo en el cjto de prueba
    best_accuracy = 0
    
    #listas de listas de cada modelo durante el entrenamiento y prueba
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    for i in range(num_experiments):
        model = constructor(*argv)
        model.to(device)

        #0.  El modelo realiza una predicción con los parámetros actuales (pesos y sesgos).
        #1. La función de pérdida calcula el error entre la predicción y las etiquetas reales.
        #2. a través de un proceso llamado retropropagación, el modelo calcula el gradiente de la función de pérdida con respecto a cada peso
        #3. Finalmente, el modelo actualiza los pesos utilizando un algoritmo de optimización, Adam, que ajusta los pesos en la dirección opuesta a los gradientes, con el objetivo de minimizar la pérdida
       

        criterion = nn.CrossEntropyLoss() # clasificación multiclase.  función de pérdida, calcula qué tan bien está funcionando el modelo durante el entrenamiento. Combina el calculo de la entropia y la funcion de activación, softmax
        #Softmax convierte las salidas del modelo en probabilidades, transformando un vector de valores (posiblemente negativos o cualquier número real) en un conjunto de probabilidades que suman 1
        #Entropía cruzada mide la diferencia entre la distribución de probabilidades predicha por el modelo y la distribución real 

        optimizer = optim.Adam(model.parameters(), weight_decay=weight_decay)#actualiza/optimiza los parámetros del modelo durante el entrenamiento para minimizar la función de pérdida
        #le pasamos los param dedl modelo para que pueda actualizar los pesos de la red neuronal 
        #weight para es evitar que los pesos del modelo crezcan demasiado, lo que puede causar sobreajuste (regular el modelo)


        train_losses.append([])
        val_losses.append([])
        train_accs.append([])
        val_accs.append([])
        
        best_acc = 0

        early_stopping_cnt = 0

        for epoch in range(1, epochs + 1):

            train_loss, train_acc = models.train_epoch(model, x_train, y_train, criterion, optimizer, epoch, 10, verbose=False) #ese 10, es que vas a procesar lotes de 10muestras?
            val_loss, val_acc = models.eval_epoch(model, x_val, y_val, criterion, 'Validation', verbose=False)

            
            train_losses[-1].append(train_loss)
            val_losses[-1].append(val_loss)
            train_accs[-1].append(train_acc)
            val_accs[-1].append(val_acc)
            
            model_name = constructor.__name__[:constructor.__name__.find('')] 
            if val_acc > best_acc: # si la precisión del cjto de val mejora, se guarda el nombre del modelo act
                best_acc = val_acc
                torch.save(model.state_dict(), f'./{model_name}model_TEMP' + experiment_name)    
                early_stopping_cnt = 0
            else:
                early_stopping_cnt += 1

            if early_stopping_cnt >= early_stopping_limit: #evitar el sobreajuste
                break

#Después de finalizar el entrenamiento con las epocas, se carga el mejor modelo 
        model.load_state_dict(torch.load(f'./{model_name}model_TEMP' + experiment_name))
    
        #
        accuracies.append(get_accuracy_by_cases(model, x_test, y_test, test_original)) #Se evalúa el modelo en el conjunto de prueba 
     #   precisions.append(get_precision_by_cases(model, x_test, y_test, test_original))
      #  recalls.append(get_recall_by_cases(model, x_test, y_test, test_original))
       # f1_scores.append(get_f1_by_cases(model, x_test, y_test, test_original))
        
        wrong_preds.append(get_wrong_predictions(model, x_test, y_test, test_original))

#se guarda el modelo con mejor precision en el co¡jto de prueba
        if accuracies[-1]['Overall'] > best_accuracy:
            torch.save(model.state_dict(), f'./best_{model_name}model' + experiment_name)    
            best_accuracy = accuracies[-1]['Overall']

        print(i + 1, "/", num_experiments, "models trained | Current model test accuracy:", accuracies[-1]['Overall'])

    return accuracies, wrong_preds, [train_losses, val_losses, train_accs, val_accs]

In [16]:
cnn_accuracies, cnn_wrong_preds, cnn_epoch_stats = train_models(models.CNN_Model, x_train_cnn, x_val_cnn, x_test_cnn, 0.0001, data, 64, 128, 4, -1, 'keras')

C:\Users\Luna Santos\AppData\Local\Temp\ipykernel_15268\3672781297.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./{model_name}mode

1 / 6 models trained | Current model test accuracy: 0.9528535980148883
2 / 6 models trained | Current model test accuracy: 0.9379652605459057
3 / 6 models trained | Current model test accuracy: 0.9230769230769231
4 / 6 models trained | Current model test accuracy: 0.09925558312655088
5 / 6 models trained | Current model test accuracy: 0.9454094292803971
6 / 6 models trained | Current model test accuracy: 0.9181141439205955


In [17]:
x_train_lstm = data.to_lstm_format(x_train)
x_val_lstm = data.to_lstm_format(x_val)
x_test_lstm = data.to_lstm_format(x_test)
for i in range(len(x_train_lstm)):
    x_train_lstm[i] = x_train_lstm[i].to(device)
    x_val_lstm[i] = x_val_lstm[i].to(device)
    x_test_lstm[i] = x_test_lstm[i].to(device)

In [18]:
lstm_accuracies, lstm_wrong_preds, lstm_epoch_stats = train_models(models.LSTM_Model, x_train_lstm, x_val_lstm, x_test_lstm, 0.0001, data, 16, 32, 8, 'keras')

C:\Users\Luna Santos\AppData\Local\Temp\ipykernel_15268\3672781297.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./{model_name}mode

1 / 6 models trained | Current model test accuracy: 0.9404466501240695
2 / 6 models trained | Current model test accuracy: 0.9404466501240695
3 / 6 models trained | Current model test accuracy: 0.9578163771712159
4 / 6 models trained | Current model test accuracy: 0.5210918114143921
5 / 6 models trained | Current model test accuracy: 0.9181141439205955
6 / 6 models trained | Current model test accuracy: 0.9429280397022333


In [12]:
x_train_normal = data.to_conv_format(x_train)
x_val_normal = data.to_conv_format(x_val)
x_test_normal = data.to_conv_format(x_test)
for i in range(len(x_train_normal)):
    x_train_normal[i] = x_train_normal[i].to(device)
    x_val_normal[i] = x_val_normal[i].to(device)
    x_test_normal[i] = x_test_normal[i].to(device)

In [13]:
deepsetv2_accuracies, deepsetv2_wrong_preds, deepsetv2_epoch_stats = train_models(models.DEEPSETV2_Model, x_train_normal, x_val_normal, x_test_normal, 0.0001, data, 32, 8, 'keras')

C:\Users\Luna Santos\AppData\Local\Temp\ipykernel_11576\3672781297.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./{model_name}mode

1 / 6 models trained | Current model test accuracy: 0.4317617866004963
2 / 6 models trained | Current model test accuracy: 0.4317617866004963
3 / 6 models trained | Current model test accuracy: 0.4317617866004963
4 / 6 models trained | Current model test accuracy: 0.4317617866004963
5 / 6 models trained | Current model test accuracy: 0.4317617866004963
6 / 6 models trained | Current model test accuracy: 0.4317617866004963


In [14]:
feedforward_accuracies, feedforward_wrong_preds, feedforward_epoch_stats = train_models(models.FEEDFORWARD_Model, x_train_normal, x_val_normal, x_test_normal, 0.0001, data, 128, 32, 8, 'keras')

C:\Users\Luna Santos\AppData\Local\Temp\ipykernel_11576\3672781297.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./{model_name}mode

1 / 6 models trained | Current model test accuracy: 0.5682382133995038
2 / 6 models trained | Current model test accuracy: 0.4739454094292804
3 / 6 models trained | Current model test accuracy: 0.5781637717121588
4 / 6 models trained | Current model test accuracy: 0.5384615384615384
5 / 6 models trained | Current model test accuracy: 0.5310173697270472
6 / 6 models trained | Current model test accuracy: 0.4838709677419355
